**Introduction:**

In this Task , Data  will be scraped from a wikipaedia page and Neighborhood segmented.

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [7]:
!pip install folium

     |████████████████████████████████| 93 kB 3.5 MB/s  eta 0:00:01


In [8]:
import folium # map rendering library

In [16]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
df = dfs[0]
df1 = df[['Postal Code', 'Borough', 'Neighbourhood']]
df1.head(12)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


**Group Neighbourhoods by Postal Code and Bursough

In [18]:
df2 = df1.groupby(["Postal Code", "Borough"])["Neighbourhood"].apply(", ".join).reset_index()
df2.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


**Obtain Latitude and Longitude from CSV**

In [20]:
url2 = "https://cocl.us/Geospatial_data"
dfgeo = pd.read_csv(url2)

dfgeo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


**Merge dfgeo and df2 dataframes**

In [22]:
df_Final = pd.merge(df2, dfgeo, how='left', left_on = 'Postal Code', right_on = 'Postal Code')
# remove repeated "Postal Code" column
df_Final.drop("Postal Code", axis=1, inplace=True)
df_Final.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Not assigned,Not assigned,NaN,NaN
1,Scarborough,"Malvern, Rouge",43.806686,-79.194353
2,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
3,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
4,Scarborough,Woburn,43.770992,-79.216917
